In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
from aimagic import *
import re, types, xml.etree.ElementTree as ET
from functools import partial
from ansi2html import Ansi2HTMLConverter

In [ ]:
def strip_ansi_codes(text):
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

In [ ]:
m = create_magic(models[0])

In [ ]:
%ai reset

In [ ]:
from execnb import nbio
import json, os
from IPython import display
from fastcore.utils import *
from fastcore.xml import *

In [ ]:
%%aio 0
Tell me a detailed step by step plan to open a Jupyter Notebook with execnb, and create a rendered HTML version of it, without using nbformat or anything similar -- instead doing it by hand from scratch.

-  Load the notebook JSON data from a file: `nb_data = json.load(open('notebook.ipynb'))`
-  Initialize an empty list to store the HTML output: `html_output = []`
-  Create a basic HTML page, including a javascript import of a markdown rendering library
-  Iterate through each cell in the notebook data:
    -  If the cell is a code cell, convert to an HTML representation (we will assume the outputs are already saved in the notebook)
    -  If the cell is a markdown cell, output it in a format the js md library can render
-  Combine the HTML output from each cell into a single string
-  Write the HTML string to a file: `with open('notebook.html', 'w') as f: f.write(html_output)`

In [ ]:
%%aip 0
Do the 1st 2 steps.

In [ ]:
fname = Path.home()/'git/nbdev/tests/minimal.ipynb'
nb = nbio.read_nb(fname)

In [ ]:
nb

```json
{ 'cells': [ { 'cell_type': 'markdown',
               'idx_': 0,
               'metadata': {},
               'source': '## A minimal notebook'},
             { 'cell_type': 'code',
               'execution_count': None,
               'idx_': 1,
               'metadata': {},
               'outputs': [ { 'data': {'text/plain': ['2']},
                              'execution_count': None,
                              'metadata': {},
                              'output_type': 'execute_result'}],
               'source': '# Do some arithmetic\n1+1'}],
  'metadata': { 'kernelspec': { 'display_name': 'Python 3 (ipykernel)',
                                'language': 'python',
                                'name': 'python3'}},
  'nbformat': 4,
  'nbformat_minor': 4,
  'path_': '/Users/jhoward/git/nbdev/tests/minimal.ipynb'}
```

In [ ]:
%%ai 0
Great. Show me the script to auto-render cells with a certain class using `marked`. If my app creates cells of that class, I want them auto-formatted.

Place a script block with this at the end of your page:

```js
  const contentDiv = document.getElementById('content');
  const renderMarkdown = (block) => { block.innerHTML = marked.parse(block.innerHTML) };
  const observer = new MutationObserver((mutations) => {
    mutations.forEach((mutation) => {
      if (mutation.type === 'childList') {
        mutation.addedNodes.forEach((node) => {
          if (node.nodeType === Node.ELEMENT_NODE && node.classList.contains('markdown'))
          renderMarkdown(node);
        });
      }
    });
  });
  observer.observe(contentDiv, { childList: true, subtree: true });
  contentDiv.querySelectorAll('.markdown').forEach(renderMarkdown);
```

In [ ]:
%%ai 0
I've written a simple HTML generation library -- here's a sample; can you see the basic idea of how to use it?

```python
async def get_list(done: bool|None = None) -> str:
    todos = TODO_LIST if done is None else [
        item for item in TODO_LIST if item.done == done]
    tlist = Ul(Li(item.tag()) for item in todos)
    elems = [H1('TODO list'), tlist, Div(id='current-todo')]
    htmxscr = Script(src="https://unpkg.com/htmx.org@1.9.12",
                     crossorigin="anonymous")
    head = [Title('TODO list'), htmxscr]
    res = Html(Head(*head), Body(*elems))
    return to_xml(res)
```

Yes, I can see the basic idea of how to use your HTML generation library. Here's a breakdown:

- You have defined functions that represent the usual HTML elements, like `Ul`, `Li`, `Div`, `Script`, `Head`, `Body`, and `Html`
- These functions take the element's content as positional arguments, which can be strings or other elements
- Attributes can be specified as keyword arguments, like `Div(id='current-todo')`
- The `Html` element represents the entire document, taking the `Head` and `Body` as arguments
- The `to_xml` function to renders the HTML element tree into an HTML string.

In [ ]:
cell = nb.cells[1]
cell

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'idx_': 1,
  'metadata': {},
  'outputs': [ { 'data': {'text/plain': ['2']},
                 'execution_count': None,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': '# Do some arithmetic\n1+1'}
```

In [ ]:
%%aip 0
Use that to write a python function that converts a code cell (such as the one in `nb` above) into a sensible HTML representation, such as a div with class `cell` containing `input` and `output` divs.

Create a div for each output, give each div the same class as the output type, and for `execute_result` type outputs show markdown (with appropriate class) if it exists, or text/plain (in Pre) otherwise.

In [ ]:
def _out_pre(o): return Pre(strip_ansi_codes('\n'.join(o)).strip())

def out_data(o):
    if o['output_type'] in ('execute_result', 'display_data'):
        dat = o['data']
        if 'text/html' in dat:
            return Div(NotStr('\n'.join(dat['text/html'])), _class='raw')
        if 'text/markdown' in dat:
            return Div('\n'.join(dat['text/markdown']), _class='markdown')
        return _out_pre(dat['text/plain'])
    return _out_pre(o['text'])

def cell_code(cell):
    return Div(
        Pre(Code(cell.source, _class="language-python"), _class='input'),
        *[Div(out_data(o), _class=o['output_type'] + " output") for o in cell.outputs],
        _class='cell cell-code'
    )

In [ ]:
print(to_xml(cell_code(cell)))

<div class="cell cell-code">
  <pre class="input">
    <code class="language-python">
# Do some arithmetic
1+1
    </code>
  </pre>
  <div class="execute_result output">
    <pre>
2
    </pre>
  </div>
</div>



In [ ]:
cell = nb.cells[0]
cell

```json
{ 'cell_type': 'markdown',
  'idx_': 0,
  'metadata': {},
  'source': '## A minimal notebook'}
```

In [ ]:
%%aip 0
Write `cell_md` now that outputs a div with the 'markdown' class, using the above example input.

In [ ]:
def cell_md(cell): return Div(cell.source, _class='cell cell-md markdown')

In [ ]:
print(to_xml(cell_md(cell)))

<div class="cell cell-md markdown">
## A minimal notebook
</div>



In [ ]:
%%aip 0
Great. Put it altogether now, taking `nb` as input, and generating a complete HTML page. Don't worry about including the markdown script we discussed -- I'll copy that in myself.

In [ ]:
def nb_html(nb, nm='Notebook'):
    head = [Title(nb.get('name', nm))]
    body = [(cell_code,cell_md)[cell.cell_type=='markdown'](cell)
            for cell in nb.cells if cell.cell_type in ('markdown','code')]
    return Html(Head(*head), Body(*body))

In [ ]:
print(to_xml(nb_html(nb)))

<html>
  <head>
    <title>
Notebook
    </title>
  </head>
  <body>
    <div class="cell cell-md markdown">
## A minimal notebook
    </div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python">
# Do some arithmetic
1+1
        </code>
      </pre>
      <div class="execute_result output">
        <pre>
2
        </pre>
      </div>
    </div>
  </body>
</html>



In [ ]:
nb.cells[11]

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'id': '933fef8f',
  'idx_': 11,
  'metadata': {},
  'outputs': [ { 'data': { 'image/png': 'iVBORw0KGgoAAAANSUhEUgAAAI4AAABXCAYAAADSzH3vAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAAJiUlEQVR4nO2de3BU1R3HPz/yIDyDITxDQh481IRCIAI+KAGtRXREqlIf6NRqtVI62j/6mLZTp9ax0xm1MyQiYKGW+ujUqq21WKtuAogoJigPiZrNSxcCcWMgJCGv3V//2MWhNJjNzc3u3c35zOxMsvfsPb+T85mbPXe/yRFVxWDoK0MiXYAhOjHiGCxhxDFYwohjsIQRx2AJI47BEr2KIyLpIlIiIhUi8qGI3BeOwgzORnq7jyMik4BJqrpXREYB5cB1qnroXK9JTU3VzMxMWws1hIfy8nKvqo7rrV18bw1UtR6oD359UkQqgDTgnOJkZmZSVlbWh3INkcDnV+KGyP88JyJ1oby2V3HOOmkmkA+825fXGZyFu6GF9SVuvK2dbP3ufEvnCFkcERkJvADcr6rNPRy/G7gbICMjw1IxhoHlo6PNFLvc/OtAPUnxcdy6IINun5/4uL6vkUISR0QSCEjzjKq+2FMbVd0EbAIoKCgwH4A5iAOeExS5KvnPoWOMHBrPvYtzuPOyLMaOHGr5nL2KIyICbAYqVPUxyz0Zwk55XRPFrkpKPv6c0Unx3Hf5dO64NJMxwxP7fe5QrjiXArcBB0Tkg+BzP1fVbf3u3WA7qso71V9QXFLJLncjKSMS+fE3Z3L7xVMZlZRgWz+hrKreAqS3dobIoqrsrPRS5KrkvdomUkcO5RfLL+DWhRkMT+zTGigk7D+jIayoKm9WNFBU4mbfZ8eZlJzEr6/N5dsXpZOUEDdg/RpxohS/X3ntw6MUudwcqm9mynnDeHjlLK6fl8bQ+IET5jRGnCjD51de2X+EYpebyoYWslNH8MiNs1kxZzIJFpbVVjHiRAldPj8vvX+YJ0qrqPG2MmPCSNbdnM/Vsyb9393fcGDEcTgd3T7+Vu7hidIqPE2nyJ08mg2r53LlhRMZEgFhTmPEcSjtXT6e2/MpG7dXc7S5nTnpY3hwRS5LZo4ncGstshhxHEZrRzfPvFvHph01eFs6mJ+VwiM3zubSaWMdIcxpjDgOobm9i61v17L5rRqa2rpYND2VtUvyWZA9NtKl9YgRJ8Icb+tky1s1/PHtWk62d7P0/PGsXTqNuRnnRbq0r8SIEyG8LR38YWcNf95dS2unj2W5E1m7dBp5acmRLi0kjDhh5lhzOxu3V/Psnjo6uv1c87XJrF0yjZkTR0W6tD5hxAkTnqY2Nmyv4q/vefCpct2cNNYsySFn3MhIl2YJI84AU9fYyvqSKl7Y60EEbpg3hXsXTyNj7PBIl9YvjDgDhLuhhcdL3Pzjg8PExw3h1gUZ3LM4h8ljhkW6NFsw4thMRX0zxSVutgXjmXdelsX3FmUzfnRSpEuzFSOOTRzwnGCdq5LXbYxnOhkjTj8pr/uCIpeb0mA88/4rpnPHJVkkD7cvbedEjDgWUFV2VzdS7HLzdlUgnvmTZTO5baG98UwnY8TpA6rKjkovRW9WUlbXxLhRQ/nl1Rdwy4KBiWc6mcE1WouoKm9UNFDsqmSf5wSTk5N4cEUuqwoGNp7pZIw4X4Hfr/w7GM+sqG8mPWUYv/3WLK6fO4XE+MH9jz6MOD3Q7fPzyv56ikvcuBtayB43gkeD8Uwrf/UYixhxzqDL5+elvYdZX+qmtrGNmRNGUXRzPssjFM90MkYcAvHM58sC8czDx0+RlzaaDavnceWFEyIaz3Qyg1qcU53BeOaOKo41d5CfMYaHrsujcOY4R6XtnMigFKe1o5un36njyZ3VeFs6WZCVwmOr5nBJjrPimU5mUInT3N7Fn3bVsnlXDceD8cwfLp3O/KyUSJcWdQwKcZpaO9myq4angvHMy4PxzHyHxzOdTEyL423p4Mmd1Ty9u47WTh9X5U3kB0uiJ57pZGJSnKMn2tm4o4rn9nxK5+l45tJpzJgQXfFMJxNT4nia2niitIrnywLxzJX5aawpzCE7SuOZTiYmxKn1trK+1M2Lew8H45nprCnMIT0luuOZTiaqxXE3nKTY5eblfUdIiBvC6oVTuWdxNpOSYyOe6WSiUpxDR5p5vMTNtoP1DEuI465F2dy1KIvxo2Irnulkokqc/Z7jrHvTzRsVgXjmmsIc7rwsm5QR/f9niIa+ERXilNUG4pnbP/mc5GEJ/OiKGXznksyYj2c6GceKo6rsrmqkyOVmd3UjY0ck8tNl57N6YcagiWc6GceJo6ps/+RzilxuyuuaGD+I45lOxjEzoaq8fugYxSVu9gfjmb9ZkcuNgzie6WQiLo7fr7x68ChFrko+OnqSjJTh/O76WazMN/FMJxMxcbp9fv65/wiPl1R9Gc98bNVsrp1t4pnRQCh7OWwBrgEaVDWvvx12dvt56X0P60urqGts4/yJoyi+JZ+r8kw8M5oI5YrzFFAMbO1PR+1dPp4v97AhGM+clZbMxtvm8Y0LTDwzGgllL4cdwQ3OLHGq08ezez5lUzCeOTdjDA+tzKNwholnRjO2vcc510Zn3pYOHt5WwUWZ5/H7VXO42MQzY4JeN3OFL7dUfCXU9zgFBQV65p6ctd5WMlNHWK3REEZEpFxVC3prF5bli5Em9hiQ5Xh5ebm3h91kUwHvQPQXZmJlHNDzWKaG8sJQ9h1/DigMdnIMeEBVN/e1QhEpC+US6HRiZRzQv7GEsqq62cqJDbGNuUVrsEQ4xdkUxr4GklgZB/RjLCEtxw2GszG/qgyWsFUcEVkmIh+LiFtEftbDcRGRdcHj+0Vkrp3920kIYykUkRMi8kHw8atI1NkbIrJFRBpE5OA5jlubE1W15QHEAVVANpAI7AMuPKvNcuBVAvuYLwTetat/Ox8hjqWQwN30iNfby1i+DswFDp7juKU5sfOKMx9wq2q1qnYCfwFWnNVmBbBVA7wDjBGRSTbWYBehjCUqUNUdwBdf0cTSnNgpThrw2Rnfe4LP9bWNEwi1zotFZJ+IvCoiueEpzXYszYmdHzn09JH32Uu2UNo4gVDq3AtMVdUWEVkO/B2YPtCFDQCW5sTOK44HSD/j+ynAEQttnECvdapqs6q2BL/eBiSISGr4SrQNS3NipzjvAdNFJEtEEoGbgJfPavMycHvwnfxC4ISq1ttYg130OhYRmSjBYJGIzCfws2wMe6X9x9Kc2ParSlW7RWQt8BqBVckWVf1QRL4fPL4B2EbgXbwbaAPusKt/OwlxLDcA94pIN3AKuEnVeXdTz/yQWkQ8wANAAvRvTsydY4MlzJ1jgyWMOAZLGHEMljDiGCxhxDFYwohjsIQRx2AJI47BEv8F1mDUv1KZDdEAAAAASUVORK5CYII=',
                           'text/plain': ['<Figure size 144x72 with 1 Axes>']},
                 'metadata': {'needs_background': 'light'},
                 'output_type': 'display_data'}],
  'source': '#|notest\nplt.figure(figsize=(2,1))\nplt.plot([1,2]);'}
```

In [ ]:
fname = Path.home()/'git/nbdev/tests/docs_test.ipynb'
nb = nbio.read_nb(fname)
print(to_xml(nb_html(nb)))

<html>
  <head>
    <title>
Notebook
    </title>
  </head>
  <body>
    <div class="cell cell-md markdown">
# a title
&gt; A description
- key1: value1
- key2: value2
- categories: [c1, c2]
    </div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python">
#|hide
#|default_exp foobar
        </code>
      </pre>
    </div>
    <div class="cell cell-md markdown">
some md
    </div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python">
#|export
import re
        </code>
      </pre>
    </div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python">
from nbdev.showdoc import show_doc
        </code>
      </pre>
    </div>
    <div class="cell cell-code">
      <pre class="input">
        <code class="language-python">
print(&#x27;\033hello&#x27;)
        </code>
      </pre>
      <div class="stream output">
        <pre>
hello
        </pre>
      </div>
    <

### TODO

- Display piccies

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()